In [ ]:
# 02 - Variography (Dashboard monobloque)
# Objetivo: UX tipo supervisor para variografía direccional y radial.

from __future__ import annotations

import json
import math
import hashlib
from pathlib import Path
from typing import Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

from src.variography import experimental_variogram_2d

# -----------------------------
# A) Setup + state
# -----------------------------
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

OUTPUTS = {
    "figures": ROOT / "outputs" / "figures",
    "tables": ROOT / "outputs" / "tables",
    "models": ROOT / "outputs" / "models",
    "logs": ROOT / "outputs" / "logs",
}
for path in OUTPUTS.values():
    path.mkdir(parents=True, exist_ok=True)

state: Dict[str, object] = {
    "df_raw": None,
    "df_work": None,
    "df_hash": None,
    "columns": {},
    "radial_scores": None,
    "theta_main": 0,
    "last_warning": "",
}

cache: Dict[Tuple[str, Tuple[Tuple[str, object], ...]], Dict[str, np.ndarray]] = {}


def df_hash(df: pd.DataFrame) -> str:
    """Stable hash for caching based on content and index."""
    hashed = pd.util.hash_pandas_object(df, index=True).values.tobytes()
    return hashlib.sha1(hashed).hexdigest()


def get_cache_key(df_key: str, params: Dict[str, object]) -> Tuple[str, Tuple[Tuple[str, object], ...]]:
    items = tuple(sorted(params.items(), key=lambda x: x[0]))
    return (df_key, items)


def log_message(message: str) -> None:
    log_path = OUTPUTS["logs"] / "variography.log"
    log_path.write_text(message + "\n", encoding="utf-8")




# -----------------------------
# Helper: experimental variogram (2D)
# -----------------------------

def compute_variogram_2d(df: pd.DataFrame, xcol: str, ycol: str, vcol: str, params: Dict[str, float]) -> Dict[str, np.ndarray]:
    key = get_cache_key(state["df_hash"], {**params, "xcol": xcol, "ycol": ycol, "vcol": vcol})
    if key in cache:
        return cache[key]
    exp = experimental_variogram_2d(df, xcol, ycol, vcol, params)
    cache[key] = exp
    return exp


# -----------------------------
# Helper: simplified vertical variogram (1D on Z)
# -----------------------------

def compute_variogram_vertical(df: pd.DataFrame, zcol: str, vcol: str, max_range: float, n_lags: int) -> Dict[str, np.ndarray]:
    params = {"lag": max_range / max(n_lags, 1), "nlag": n_lags}
    key = get_cache_key(state["df_hash"], {**params, "zcol": zcol, "vcol": vcol, "mode": "vertical"})
    if key in cache:
        return cache[key]

    z = df[zcol].to_numpy(dtype=float)
    v = df[vcol].to_numpy(dtype=float)
    n = len(df)
    if n < 2:
        lags = np.zeros(n_lags)
        gamma = np.zeros(n_lags)
        npp = np.zeros(n_lags, dtype=int)
        exp = {"lags": lags, "gamma": gamma, "npp": npp, "params": params}
        cache[key] = exp
        return exp

    dz = np.abs(z[:, None] - z[None, :])
    dv = (v[:, None] - v[None, :]) ** 2
    iu = np.triu_indices(n, k=1)
    dz = dz[iu]
    dv = dv[iu]

    bins = np.linspace(0, max_range, n_lags + 1)
    lags = 0.5 * (bins[:-1] + bins[1:])
    gamma = np.zeros(n_lags)
    npp = np.zeros(n_lags, dtype=int)

    for i in range(n_lags):
        mask = (dz >= bins[i]) & (dz < bins[i + 1])
        npp[i] = int(mask.sum())
        gamma[i] = 0.5 * float(dv[mask].mean()) if npp[i] > 0 else np.nan

    exp = {"lags": lags, "gamma": gamma, "npp": npp, "params": params}
    cache[key] = exp
    return exp


# -----------------------------
# B) Load CSV + map columns
# -----------------------------

def parse_upload(upload_widget: widgets.FileUpload) -> Optional[pd.DataFrame]:
    if not upload_widget.value:
        return None
    content = next(iter(upload_widget.value.values()))
    data = content["content"]
    return pd.read_csv(pd.io.common.BytesIO(data))


def apply_mapping(
    df: pd.DataFrame,
    xcol: str,
    ycol: str,
    zcol: Optional[str],
    grade_col: str,
    domain_col: Optional[str],
) -> Tuple[pd.DataFrame, str]:
    cols = [xcol, ycol, grade_col]
    if zcol:
        cols.append(zcol)
    if domain_col:
        cols.append(domain_col)

    df_work = df[cols].copy()
    for col in [xcol, ycol, grade_col] + ([zcol] if zcol else []):
        df_work[col] = pd.to_numeric(df_work[col], errors="coerce")

    df_work = df_work.dropna(subset=[xcol, ycol, grade_col] + ([zcol] if zcol else []))
    dup_count = df_work.duplicated(subset=[xcol, ycol] + ([zcol] if zcol else [])).sum()
    warning = f"Duplicados en coordenadas: {dup_count}" if dup_count > 0 else "Sin duplicados en coordenadas."
    return df_work, warning


# -----------------------------
# C) Variography controls
# -----------------------------
file_upload = widgets.FileUpload(accept=".csv", multiple=False, description="CSV")

x_dropdown = widgets.Dropdown(options=[], description="X")
y_dropdown = widgets.Dropdown(options=[], description="Y")
z_dropdown = widgets.Dropdown(options=[("–", None)], description="Z")
grade_dropdown = widgets.Dropdown(options=[], description="GRADE")
domain_dropdown = widgets.Dropdown(options=[("–", None)], description="DOMAIN")

apply_button = widgets.Button(description="Aplicar mapeo", button_style="primary")

max_range = widgets.FloatText(value=500.0, description="max_range")
use_lag_size = widgets.Checkbox(value=False, description="usar lag_size")
lag_size = widgets.FloatText(value=50.0, description="lag_size")
n_lags = widgets.IntText(value=12, description="n_lags")

bandwidth = widgets.FloatText(value=9999.0, description="bandwidth")
ang_tol = widgets.FloatSlider(value=22.5, min=1.0, max=90.0, step=0.5, description="ang_tol")
min_pairs = widgets.IntText(value=30, description="min_pairs")

estimator = widgets.Dropdown(options=["Matheron"], description="estimator")
use_z = widgets.Checkbox(value=False, description="use_z (vertical)")

recalc_button = widgets.Button(description="Recalcular", button_style="warning")

# Radial params
radial_dtheta = widgets.IntSlider(value=5, min=1, max=30, step=1, description="dtheta")
radial_p = widgets.FloatSlider(value=0.7, min=0.2, max=0.95, step=0.05, description="p (range)")
radial_k = widgets.IntSlider(value=3, min=2, max=6, step=1, description="k (slope)")
radial_w1 = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="w1")
radial_w2 = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="w2")
radial_w3 = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="w3")

# Directional controls
mode_selector = widgets.RadioButtons(options=["fixed", "around main"], description="grid")
select_direction = widgets.RadioButtons(options=[], description="elegir dir")

# Theta main selection
theta_slider = widgets.IntSlider(value=0, min=0, max=359, step=1, description="θ_main")

# Bin zoom
bin_start = widgets.IntSlider(value=1, min=1, max=12, step=1, description="bin_start")
bin_end = widgets.IntSlider(value=12, min=1, max=12, step=1, description="bin_end")

# Export
export_button = widgets.Button(description="Export", button_style="success")

# Feedback/output areas
status_out = widgets.Output()
radial_out = widgets.Output()
grid_out = widgets.Output()
major_out = widgets.Output()
zoom_table_out = widgets.Output()
zoom_metrics_out = widgets.Output()


# -----------------------------
# D) Radial plot (0-360)
# -----------------------------

def compute_radial_scores(df: pd.DataFrame, xcol: str, ycol: str, vcol: str, params: Dict[str, float]) -> pd.DataFrame:
    variance = float(df[vcol].var()) if len(df) > 1 else 0.0
    thetas = np.arange(0, 360, radial_dtheta.value)
    rows = []

    for theta in thetas:
        params_dir = {**params, "azm": float(theta)}
        exp = compute_variogram_2d(df, xcol, ycol, vcol, params_dir)
        h = np.asarray(exp["lags"], dtype=float)
        gamma = np.asarray(exp["gamma"], dtype=float)
        npp = np.asarray(exp["npp"], dtype=float)

        range_proxy = np.nan
        if variance > 0:
            thresh = radial_p.value * variance
            idx = np.where(gamma >= thresh)[0]
            if idx.size:
                range_proxy = h[int(idx[0])]
        if not np.isfinite(range_proxy):
            range_proxy = float(np.nanmax(h)) if np.isfinite(np.nanmax(h)) else 0.0

        k = min(radial_k.value, len(h))
        slope = float(np.polyfit(h[:k], gamma[:k], 1)[0]) if k >= 2 else 0.0

        npairs_mean = float(np.nanmean(npp)) if len(npp) else 0.0
        penalty = max(0.0, (min_pairs.value - npairs_mean) / max(min_pairs.value, 1))

        score = radial_w1.value * range_proxy - radial_w2.value * slope - radial_w3.value * penalty
        rows.append({
            "theta": float(theta),
            "score_raw": score,
            "range_proxy": range_proxy,
            "slope": slope,
            "npairs_mean": npairs_mean,
        })

    df_scores = pd.DataFrame(rows)
    if df_scores.empty:
        return df_scores

    smin = float(df_scores["score_raw"].min())
    smax = float(df_scores["score_raw"].max())
    if math.isclose(smin, smax):
        df_scores["score"] = 0.0
    else:
        df_scores["score"] = (df_scores["score_raw"] - smin) / (smax - smin)
    return df_scores


def plot_radial(df_scores: pd.DataFrame) -> None:
    radial_out.clear_output()
    with radial_out:
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection="polar")
        if df_scores.empty:
            ax.set_title("Radial scores (sin datos)")
        else:
            theta_rad = np.deg2rad(df_scores["theta"].to_numpy())
            scores = df_scores["score"].to_numpy()
            ax.plot(theta_rad, scores, "o-", color="tab:blue")
            ax.set_theta_zero_location("N")
            ax.set_theta_direction(-1)
            ax.set_ylim(0, 1.05)
            ax.set_title("Radial score 0-360°")
        plt.tight_layout()
        plt.show()


# -----------------------------
# E/F) Directionals + Major/Perp/Vertical
# -----------------------------

def plot_directional_grid(
    df: pd.DataFrame,
    xcol: str,
    ycol: str,
    vcol: str,
    params: Dict[str, float],
    theta_main_val: float,
    show: bool = True,
) -> Optional[plt.Figure]:
    fig, axes = plt.subplots(2, 2, figsize=(8, 6))
    if mode_selector.value == "fixed":
        angles = [0, 45, 90, 135]
    else:
        angles = [(theta_main_val + d) % 360 for d in [0, 45, 90, 135]]
    select_direction.options = [(f"{ang}°", ang) for ang in angles]
    for ax, ang in zip(axes.flat, angles):
        params_dir = {**params, "azm": float(ang)}
        exp = compute_variogram_2d(df, xcol, ycol, vcol, params_dir)
        h = np.asarray(exp["lags"], dtype=float)
        gamma = np.asarray(exp["gamma"], dtype=float)
        npp = np.asarray(exp["npp"], dtype=float)
        sizes = np.clip(npp, 1, None) * 3
        colors = np.where(npp < min_pairs.value, "lightgray", "tab:blue")
        ax.scatter(h, gamma, s=sizes, c=colors)
        ax.set_title(f"{ang}°")
        ax.set_xlabel("h")
        ax.set_ylabel("gamma")
    plt.tight_layout()
    if show:
        grid_out.clear_output()
        with grid_out:
            plt.show()
    return fig


def plot_major_perp_vertical(
    df: pd.DataFrame,
    xcol: str,
    ycol: str,
    zcol: Optional[str],
    vcol: str,
    params: Dict[str, float],
    theta_main_val: float,
    use_vertical: bool,
    zoom_range: Optional[Tuple[int, int]] = None,
    show: bool = True,
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], Optional[Dict[str, np.ndarray]]]:
    if use_vertical and zcol:
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    theta_major = float(theta_main_val)
    theta_perp = float((theta_main_val + 90) % 360)

    major_params = {**params, "azm": theta_major}
    perp_params = {**params, "azm": theta_perp}

    major_exp = compute_variogram_2d(df, xcol, ycol, vcol, major_params)
    perp_exp = compute_variogram_2d(df, xcol, ycol, vcol, perp_params)

    def plot_exp(ax, exp, title):
        h = np.asarray(exp["lags"], dtype=float)
        gamma = np.asarray(exp["gamma"], dtype=float)
        npp = np.asarray(exp["npp"], dtype=float)
        if zoom_range:
            start, end = zoom_range
            start = max(start - 1, 0)
            end = min(end, len(h))
            h = h[start:end]
            gamma = gamma[start:end]
            npp = npp[start:end]
        colors = np.where(npp < min_pairs.value, "lightgray", "tab:blue")
        sizes = np.clip(npp, 1, None) * 3
        ax.scatter(h, gamma, s=sizes, c=colors)
        ax.set_title(title)
        ax.set_xlabel("h")
        ax.set_ylabel("gamma")

    plot_exp(axes[0], major_exp, f"Major {theta_major:.0f}°")
    plot_exp(axes[1], perp_exp, f"Perp {theta_perp:.0f}°")

    vertical_exp = None
    if use_vertical and zcol:
        vertical_exp = compute_variogram_vertical(df, zcol, vcol, params["max_range"], params["nlag"])
        plot_exp(axes[2], vertical_exp, "Vertical")

    plt.tight_layout()
    if show:
        major_out.clear_output()
        with major_out:
            plt.show()

    return major_exp, perp_exp, vertical_exp


# -----------------------------
# G) Bin Zoom + tabla
# -----------------------------

def update_zoom_table(exp: Dict[str, np.ndarray]) -> None:
    zoom_table_out.clear_output()
    zoom_metrics_out.clear_output()
    if exp is None:
        return

    h = np.asarray(exp["lags"], dtype=float)
    gamma = np.asarray(exp["gamma"], dtype=float)
    npp = np.asarray(exp["npp"], dtype=float)

    start = max(bin_start.value - 1, 0)
    end = min(bin_end.value, len(h))
    h_zoom = h[start:end]
    gamma_zoom = gamma[start:end]
    npp_zoom = npp[start:end]

    table = pd.DataFrame({
        "bin_idx": np.arange(start + 1, end + 1),
        "h_center": h_zoom,
        "gamma": gamma_zoom,
        "npairs": npp_zoom,
    })

    slope_local = float(np.polyfit(h_zoom, gamma_zoom, 1)[0]) if len(h_zoom) >= 2 else 0.0
    npairs_min = int(np.nanmin(npp_zoom)) if len(npp_zoom) else 0
    npairs_mean = float(np.nanmean(npp_zoom)) if len(npp_zoom) else 0.0

    with zoom_table_out:
        display(table)

    with zoom_metrics_out:
        print(f"Slope local: {slope_local:.4f}")
        print(f"npairs_min: {npairs_min}")
        print(f"npairs_mean: {npairs_mean:.2f}")


# -----------------------------
# H) Export outputs
# -----------------------------

def plot_exp_to_file(exp: Dict[str, np.ndarray], title: str, path: Path) -> None:
    fig, ax = plt.subplots(figsize=(6, 4))
    h = np.asarray(exp["lags"], dtype=float)
    gamma = np.asarray(exp["gamma"], dtype=float)
    npp = np.asarray(exp["npp"], dtype=float)
    colors = np.where(npp < min_pairs.value, "lightgray", "tab:blue")
    sizes = np.clip(npp, 1, None) * 3
    ax.scatter(h, gamma, s=sizes, c=colors)
    ax.set_title(title)
    ax.set_xlabel("h")
    ax.set_ylabel("gamma")
    fig.tight_layout()
    fig.savefig(path, dpi=150)
    plt.close(fig)


def export_outputs(
    df_scores: pd.DataFrame,
    major_exp: Dict[str, np.ndarray],
    perp_exp: Dict[str, np.ndarray],
    df_work: pd.DataFrame,
    columns: Dict[str, Optional[str]],
    params: Dict[str, float],
    theta_main_val: float,
) -> None:
    if df_scores is not None:
        df_scores.to_csv(OUTPUTS["tables"] / "radial_scores.csv", index=False)

    def exp_to_df(exp: Dict[str, np.ndarray]) -> pd.DataFrame:
        return pd.DataFrame({
            "h_center": exp["lags"],
            "gamma": exp["gamma"],
            "npairs": exp["npp"],
        })

    if major_exp is not None:
        exp_to_df(major_exp).to_csv(OUTPUTS["tables"] / "variogram_major.csv", index=False)
        plot_exp_to_file(major_exp, f"Major {theta_main_val:.0f}°", OUTPUTS["figures"] / "variogram_major.png")
    if perp_exp is not None:
        exp_to_df(perp_exp).to_csv(OUTPUTS["tables"] / "variogram_perp.csv", index=False)
        plot_exp_to_file(perp_exp, f"Perp {(theta_main_val + 90) % 360:.0f}°", OUTPUTS["figures"] / "variogram_perp.png")

    # Radial score figure
    if df_scores is not None:
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection="polar")
        if not df_scores.empty:
            theta_rad = np.deg2rad(df_scores["theta"].to_numpy())
            scores = df_scores["score"].to_numpy()
            ax.plot(theta_rad, scores, "o-", color="tab:blue")
            ax.set_theta_zero_location("N")
            ax.set_theta_direction(-1)
            ax.set_ylim(0, 1.05)
        ax.set_title("Radial score 0-360°")
        fig.tight_layout()
        fig.savefig(OUTPUTS["figures"] / "radial_score.png", dpi=150)
        plt.close(fig)

    # Directional grid figure
    grid_fig = plot_directional_grid(
        df_work,
        columns["x"],
        columns["y"],
        columns["grade"],
        params,
        theta_main_val,
        show=False,
    )
    if grid_fig:
        grid_fig.savefig(OUTPUTS["figures"] / "variogram_directional_grid.png", dpi=150)
        plt.close(grid_fig)

    params_out = {
        "max_range": max_range.value,
        "n_lags": n_lags.value,
        "lag_size": lag_size.value,
        "bandwidth": bandwidth.value,
        "ang_tol": ang_tol.value,
        "min_pairs": min_pairs.value,
        "theta_main": theta_main_val,
        "radial": {
            "dtheta": radial_dtheta.value,
            "p": radial_p.value,
            "k": radial_k.value,
            "w1": radial_w1.value,
            "w2": radial_w2.value,
            "w3": radial_w3.value,
        },
    }
    (OUTPUTS["models"] / "variography_params.json").write_text(json.dumps(params_out, indent=2), encoding="utf-8")


# -----------------------------
# UI logic
# -----------------------------

def update_column_options(df: pd.DataFrame) -> None:
    options = [(col, col) for col in df.columns]
    x_dropdown.options = options
    y_dropdown.options = options
    grade_dropdown.options = options
    z_dropdown.options = [("–", None)] + options
    domain_dropdown.options = [("–", None)] + options


def on_upload_change(_):
    df = parse_upload(file_upload)
    if df is None:
        return
    state["df_raw"] = df
    update_column_options(df)
    with status_out:
        clear_output()
        print(f"CSV cargado: {df.shape[0]} filas, {df.shape[1]} columnas")


file_upload.observe(on_upload_change, names="value")


def on_apply_mapping(_):
    if state["df_raw"] is None:
        with status_out:
            clear_output()
            print("Primero cargue un CSV.")
        return
    df_work, warning = apply_mapping(
        state["df_raw"],
        x_dropdown.value,
        y_dropdown.value,
        z_dropdown.value,
        grade_dropdown.value,
        domain_dropdown.value,
    )
    state["df_work"] = df_work
    state["df_hash"] = df_hash(df_work)
    state["columns"] = {
        "x": x_dropdown.value,
        "y": y_dropdown.value,
        "z": z_dropdown.value,
        "grade": grade_dropdown.value,
        "domain": domain_dropdown.value,
    }

    diag = math.dist(
        (df_work[x_dropdown.value].min(), df_work[y_dropdown.value].min()),
        (df_work[x_dropdown.value].max(), df_work[y_dropdown.value].max()),
    )
    warn_diag = "" if max_range.value <= diag else "Advertencia: max_range excede la diagonal del bbox."

    use_z.value = bool(z_dropdown.value)
    use_z.disabled = not bool(z_dropdown.value)

    with status_out:
        clear_output()
        print(f"Datos limpios: {df_work.shape[0]} filas.")
        print(warning)
        if warn_diag:
            print(warn_diag)


apply_button.on_click(on_apply_mapping)


def resolve_lags() -> int:
    if use_lag_size.value:
        if lag_size.value <= 0:
            return n_lags.value
        return max(1, int(math.floor(max_range.value / lag_size.value)))
    return n_lags.value


def sync_bins():
    total = resolve_lags()
    n_lags.value = total
    bin_start.max = total
    bin_end.max = total
    if bin_end.value < bin_start.value:
        bin_end.value = bin_start.value


def on_param_change(_):
    sync_bins()


use_lag_size.observe(on_param_change, names="value")
lag_size.observe(on_param_change, names="value")
max_range.observe(on_param_change, names="value")


def on_select_direction_change(change):
    if change["name"] == "value" and change["new"] is not None:
        theta_slider.value = int(change["new"])


select_direction.observe(on_select_direction_change, names="value")


def summarize_theta(df_scores: pd.DataFrame, theta_main_val: float) -> None:
    if df_scores is None or df_scores.empty:
        return
    deltas = (df_scores["theta"] - float(theta_main_val)).abs()
    row = df_scores.loc[deltas.idxmin()]
    print(
        "θ={:.0f}° | score={:.3f} | range_proxy={:.2f} | slope={:.4f} | npairs_mean={:.1f}".format(
            row["theta"], row["score"], row["range_proxy"], row["slope"], row["npairs_mean"]
        )
    )


def recompute_all(_=None):
    if state["df_work"] is None:
        with status_out:
            clear_output()
            print("Cargue un CSV y aplique el mapeo.")
        return

    df_work = state["df_work"]
    xcol = state["columns"]["x"]
    ycol = state["columns"]["y"]
    vcol = state["columns"]["grade"]

    nlag = resolve_lags()
    params = {
        "lag": max_range.value / max(nlag, 1),
        "nlag": nlag,
        "azm": float(theta_slider.value),
        "atol": ang_tol.value,
        "bandwh": bandwidth.value,
        "max_range": max_range.value,
    }

    df_scores = compute_radial_scores(df_work, xcol, ycol, vcol, params)
    state["radial_scores"] = df_scores
    plot_radial(df_scores)
    update_theta_views()


def update_theta_views(_=None):
    if state["df_work"] is None:
        return

    if state.get("radial_scores") is None:
        recompute_all()
        return

    df_work = state["df_work"]
    xcol = state["columns"]["x"]
    ycol = state["columns"]["y"]
    zcol = state["columns"]["z"]
    vcol = state["columns"]["grade"]

    nlag = resolve_lags()
    params = {
        "lag": max_range.value / max(nlag, 1),
        "nlag": nlag,
        "azm": float(theta_slider.value),
        "atol": ang_tol.value,
        "bandwh": bandwidth.value,
        "max_range": max_range.value,
    }

    plot_directional_grid(df_work, xcol, ycol, vcol, params, theta_slider.value)

    major_exp, perp_exp, vertical_exp = plot_major_perp_vertical(
        df_work,
        xcol,
        ycol,
        zcol,
        vcol,
        params,
        theta_slider.value,
        use_z.value,
        zoom_range=(bin_start.value, bin_end.value),
    )

    update_zoom_table(major_exp)

    with status_out:
        clear_output()
        print("---")
        summarize_theta(state.get("radial_scores"), theta_slider.value)
        major_range = float(np.nanmax(major_exp["lags"])) if major_exp else 0.0
        perp_range = float(np.nanmax(perp_exp["lags"])) if perp_exp else 0.0
        if perp_range > 0:
            print(f"Anisotropía proxy (major/perp): {major_range / perp_range:.2f}")
        if vertical_exp is not None:
            vert_range = float(np.nanmax(vertical_exp["lags"])) if vertical_exp else 0.0
            if vert_range > 0:
                print(f"Anisotropía proxy (major/vertical): {major_range / vert_range:.2f}")

        npairs_avg = float(np.nanmean(major_exp["npp"])) if major_exp else 0.0
        if npairs_avg < min_pairs.value:
            print("Advertencia: npairs promedio bajo en major.")

    state["last_major_exp"] = major_exp
    state["last_perp_exp"] = perp_exp

recalc_button.on_click(recompute_all)


theta_slider.observe(update_theta_views, names="value")
mode_selector.observe(update_theta_views, names="value")
bin_start.observe(update_theta_views, names="value")
bin_end.observe(update_theta_views, names="value")


def on_export(_):
    if state["df_work"] is None:
        with status_out:
            print("No hay datos para exportar.")
        return

    nlag = resolve_lags()
    params = {
        "lag": max_range.value / max(nlag, 1),
        "nlag": nlag,
        "azm": float(theta_slider.value),
        "atol": ang_tol.value,
        "bandwh": bandwidth.value,
        "max_range": max_range.value,
    }

    export_outputs(
        state.get("radial_scores"),
        state.get("last_major_exp"),
        state.get("last_perp_exp"),
        state["df_work"],
        state["columns"],
        params,
        theta_slider.value,
    )
    with status_out:
        print("Export completado en outputs/.")


export_button.on_click(on_export)


# -----------------------------
# Layout: dashboard
# -----------------------------
controls_left = widgets.VBox([
    widgets.HTML("<b>A) Setup + state</b>"),
    file_upload,
    x_dropdown,
    y_dropdown,
    z_dropdown,
    grade_dropdown,
    domain_dropdown,
    apply_button,
    widgets.HTML("<b>C) Variography controls</b>"),
    max_range,
    use_lag_size,
    lag_size,
    n_lags,
    bandwidth,
    ang_tol,
    min_pairs,
    estimator,
    use_z,
    recalc_button,
    widgets.HTML("<b>D) Radial params</b>"),
    radial_dtheta,
    radial_p,
    radial_k,
    radial_w1,
    radial_w2,
    radial_w3,
    widgets.HTML("<b>E) Directional grid</b>"),
    mode_selector,
    select_direction,
    widgets.HTML("<b>F) Major/Perp</b>"),
    theta_slider,
    widgets.HTML("<b>G) Bin Zoom</b>"),
    bin_start,
    bin_end,
    widgets.HTML("<b>H) Export</b>"),
    export_button,
])

center_plots = widgets.VBox([
    widgets.HTML("<b>D) Radial plot 0-360°</b>"),
    radial_out,
    widgets.HTML("<b>E/F) Directional grid + Major/Perp/Vertical</b>"),
    grid_out,
    major_out,
])

bottom_panel = widgets.VBox([
    widgets.HTML("<b>G) Bin table + feedback</b>"),
    zoom_table_out,
    zoom_metrics_out,
    widgets.HTML("<b>Logs/Status</b>"),
    status_out,
])

layout = widgets.VBox([
    widgets.HBox([controls_left, center_plots]),
    bottom_panel,
])

display(layout)
